In [105]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os 
import re
from tqdm import tqdm

In [2]:
df1 = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/data_joining/anime.csv", sep = "\t")
df2 = pd.read_json("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/fin_anime_dfv1.json")

In [3]:
df3 = df1.merge(df2, left_on='anime_id', right_on= 'anime_id', how = "inner")

In [47]:
print(df2.loc[8000,['image','Image URL']])

image        https://m.media-amazon.com/images/M/MV5BODI5Nm...
Image URL    https://cdn.myanimelist.net/images/anime/2/271...
Name: 8000, dtype: object


In [26]:
df3['new_anime_image_link'] = df3['anime_url'].apply(lambda x: x  + "/pics")

In [31]:
url = df3['new_anime_image_link'][800]

In [32]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

In [33]:
soup


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html class="appearance-none" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<link crossorigin="anonymous" href="//fonts.gstatic.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//fonts.googleapis.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//tags-cdn.deployads.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//www.googletagservices.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//www.googletagmanager.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//apis.google.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//pixel-sync.sitescout.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//pixel.tapad.com/" rel="preconnect"/>
<link crossorigin="anonymous" href="//c.deployads.com/" rel="preconnect"/>
<link crossorigin="anonymous" hre

In [34]:
df = pd.read_csv("/Users/justinvhuang/Desktop/CSE-6242-Group-Project/data_joining/anime-dataset-2023.csv")

In [42]:
image_urls = extract_image_urls(df3['new_anime_image_link'][800])

/var/folders/vz/cyjrzm0x74q21sb72nhkdfmr0000gn/T/ipykernel_37629/2240241191.py:3: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  soup = BeautifulSoup(html, 'html.parser')


In [48]:
import wikipedia

In [52]:
ny = wikipedia.page("Frieren")

In [57]:
import pickle

In [90]:
# Path to your pickle file
pickle_file_path = "/Users/justinvhuang/Desktop/CSE-6242-Group-Project/prompting/wiki_dict.pkl"  


# Load data from the pickle file
with open(pickle_file_path, 'rb') as file:
    loaded_data_imdb = pickle.load(file)

In [91]:
# Path to your pickle file
pickle_file_path = "/Users/justinvhuang/Desktop/CSE-6242-Group-Project/data_joining/anime_wiki_dict.pkl"  


# Load data from the pickle file
with open(pickle_file_path, 'rb') as file:
    loaded_data_mal = pickle.load(file)

In [92]:
joined_dict = {**loaded_data_imdb, **loaded_data_mal}


In [96]:
new_dict = {key: value['Title'] for key, value in joined_dict.items()}

In [107]:
scrap_df = pd.DataFrame.from_dict(new_dict, orient = 'index').reset_index().rename(columns={'index':'anime_id', 0:'Name'})

In [112]:
# Function to get Wikipedia content for a given title
def get_wikipedia_content(title):
    import time

    try:
        page = wikipedia.page(title)
        return page.images
    except wikipedia.exceptions.DisambiguationError as e:
        # If the title is ambiguous, you can handle it as per your requirement
        return f"Ambiguous title: {title}. Options: {e.options}"
    except wikipedia.exceptions.PageError:
        return f"No Wikipedia page found for: {title}"
    except wikipedia.exceptions.HTTPTimeoutError as e:
        return f"Timeout error occurred: {str(e)}"
    except wikipedia.exceptions.WikipediaException as e:
        if "HTTP 443" in str(e):
            print("Received HTTP 443 error. Pausing for 2 minutes...")
            time.sleep(120)  # Pause for 2 minutes (120 seconds)
            print("Resuming...")
            return get_wikipedia_content(title)  # Retry
        else:
            return f"Error occurred: {str(e)}"
    except Exception as e:
        return f"Error occurred: {str(e)}"
    
def fetch_wikipedia_content(df, get_wikipedia_content_func):
    # Dictionary to store data
    data_dict = {}

    # Loop through each title in the DataFrame with tqdm for progress tracking
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Fetching Wikipedia Content"):
        imdb_id = row['anime_id']
        title = row['Name']

        # Get Wikipedia content for the title
        content = get_wikipedia_content_func(title)

        # Add data to dictionary
        if imdb_id not in data_dict:
            data_dict[imdb_id] = {'image': content}
        else:
            print(f"Duplicate IMDb ID found: {imdb_id}")

    return data_dict

In [115]:
fetch1 = fetch_wikipedia_content(scrap_df[0:1000],get_wikipedia_content)

Fetching Wikipedia Content:   4%|▎         | 37/1000 [00:20<09:06,  1.76it/s]/opt/miniconda3/envs/spacy/lib/python3.8/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /opt/miniconda3/envs/spacy/lib/python3.8/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
Fetching Wikipedia Content: 100%|██████████| 1000/1000 [12:25<00:00,  1.34it/s] 


In [116]:
fetch2 = fetch_wikipedia_content(scrap_df[1000:2000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [15:58<00:00,  1.04it/s]


In [117]:
fetch3 = fetch_wikipedia_content(scrap_df[2000:3000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [16:38<00:00,  1.00it/s]


In [118]:
fetch4 = fetch_wikipedia_content(scrap_df[3000:4000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [19:06<00:00,  1.15s/it] 


In [119]:
fetch5 = fetch_wikipedia_content(scrap_df[4000:5000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [16:30<00:00,  1.01it/s]


In [120]:
fetch6 = fetch_wikipedia_content(scrap_df[5000:6000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [29:02<00:00,  1.74s/it]   


In [121]:
fetch7 = fetch_wikipedia_content(scrap_df[6000:7000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [17:44<00:00,  1.06s/it] 


In [122]:
fetch8 = fetch_wikipedia_content(scrap_df[7000:8000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [14:58<00:00,  1.11it/s]


In [123]:
fetch9 = fetch_wikipedia_content(scrap_df[8000:9000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [14:58<00:00,  1.11it/s]


In [124]:
fetch10 = fetch_wikipedia_content(scrap_df[9000:10000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [15:40<00:00,  1.06it/s] 


In [125]:
fetch11 = fetch_wikipedia_content(scrap_df[10000:11000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [14:06<00:00,  1.18it/s]


In [126]:
fetch12 = fetch_wikipedia_content(scrap_df[11000:12000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [14:50<00:00,  1.12it/s]


In [127]:
fetch13 = fetch_wikipedia_content(scrap_df[12000:13000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [15:00<00:00,  1.11it/s]


In [128]:
fetch14 = fetch_wikipedia_content(scrap_df[13000:14000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [16:10<00:00,  1.03it/s]


In [129]:
fetch15 = fetch_wikipedia_content(scrap_df[14000:15000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [13:53<00:00,  1.20it/s]


In [130]:
fetch16 = fetch_wikipedia_content(scrap_df[15000:16000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [13:53<00:00,  1.20it/s]


In [131]:
fetch17 = fetch_wikipedia_content(scrap_df[16000:17000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [16:33<00:00,  1.01it/s] 


In [132]:
fetch18 = fetch_wikipedia_content(scrap_df[17000:18000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [12:36<00:00,  1.32it/s]


In [133]:
fetch19 = fetch_wikipedia_content(scrap_df[18000:19000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [12:22<00:00,  1.35it/s]


In [134]:
fetch20 = fetch_wikipedia_content(scrap_df[19000:20000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [12:49<00:00,  1.30it/s]


In [135]:
fetch21 = fetch_wikipedia_content(scrap_df[20000:21000],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1000/1000 [16:23<00:00,  1.02it/s] 


In [136]:
fetch22 = fetch_wikipedia_content(scrap_df[21000:],get_wikipedia_content)

Fetching Wikipedia Content: 100%|██████████| 1214/1214 [16:56<00:00,  1.19it/s]


In [139]:
# Create an empty dictionary to store the merged result
merged_dict = {}

# Iterate over the dictionaries fetch1 to fetch22
for i in range(1, 23):
    # Assuming fetch1 to fetch22 are already defined dictionaries
    dict_name = 'fetch' + str(i)
    if dict_name in locals():
        # Merge the current dictionary with the merged_dict
        merged_dict.update(locals()[dict_name])

In [140]:
merged_dict

{'tt21030032': {'image': ['https://upload.wikimedia.org/wikipedia/commons/d/da/Wikipe-tan_face.svg',
   'https://upload.wikimedia.org/wikipedia/en/4/4a/Commons-logo.svg',
   'https://upload.wikimedia.org/wikipedia/en/1/1c/Oshi_no_Ko_Volume_1.jpg',
   'https://upload.wikimedia.org/wikipedia/en/9/96/Symbol_category_class.svg']},
 'tt1343075': {'image': ['https://upload.wikimedia.org/wikipedia/commons/d/d6/Ayako_Kawasumi_20060805_Otakon_02.jpg',
   'https://upload.wikimedia.org/wikipedia/commons/e/e9/Hiroshi_Matsuyama_20080704_Japan_Expo_03.jpg',
   'https://upload.wikimedia.org/wikipedia/commons/6/68/Logo_dotHack.svg',
   'https://upload.wikimedia.org/wikipedia/commons/e/ed/Sakurai_Takahiro_from_%22GODZILLA_The_Planet_Eater%22_at_Opening_Ceremony_of_the_Tokyo_International_Film_Festival_2018_%2831747117738%29.jpg',
   'https://upload.wikimedia.org/wikipedia/en/5/54/.hackGULogo.png',
   'https://upload.wikimedia.org/wikipedia/en/9/9f/GU_characters.jpg',
   'https://upload.wikimedia.org/wi

In [141]:
# Define a function to filter out items ending with '.svg' from a list
def filter_svg_from_list(input_list):
    return [item for item in input_list if not item.endswith('.svg')]

# Iterate over the key-value pairs in the merged_dict
for key, value in merged_dict.items():
    if isinstance(value, list):
        # Filter out items ending with '.svg' from the list
        merged_dict[key] = filter_svg_from_list(value)

In [142]:
merged_dict

{'tt21030032': {'image': ['https://upload.wikimedia.org/wikipedia/commons/d/da/Wikipe-tan_face.svg',
   'https://upload.wikimedia.org/wikipedia/en/4/4a/Commons-logo.svg',
   'https://upload.wikimedia.org/wikipedia/en/1/1c/Oshi_no_Ko_Volume_1.jpg',
   'https://upload.wikimedia.org/wikipedia/en/9/96/Symbol_category_class.svg']},
 'tt1343075': {'image': ['https://upload.wikimedia.org/wikipedia/commons/d/d6/Ayako_Kawasumi_20060805_Otakon_02.jpg',
   'https://upload.wikimedia.org/wikipedia/commons/e/e9/Hiroshi_Matsuyama_20080704_Japan_Expo_03.jpg',
   'https://upload.wikimedia.org/wikipedia/commons/6/68/Logo_dotHack.svg',
   'https://upload.wikimedia.org/wikipedia/commons/e/ed/Sakurai_Takahiro_from_%22GODZILLA_The_Planet_Eater%22_at_Opening_Ceremony_of_the_Tokyo_International_Film_Festival_2018_%2831747117738%29.jpg',
   'https://upload.wikimedia.org/wikipedia/en/5/54/.hackGULogo.png',
   'https://upload.wikimedia.org/wikipedia/en/9/9f/GU_characters.jpg',
   'https://upload.wikimedia.org/wi

In [143]:
# Iterate over each key-value pair in the dictionary
for key, value in merged_dict.items():
    # Check if 'image' key exists and value is a list
    if 'image' in value and isinstance(value['image'], list):
        # Filter out URLs ending with ".svg"
        value['image'] = [url for url in value['image'] if not url.endswith('.svg')]

In [145]:
import pickle 

In [146]:
# Save dictionary to a pickle file
with open('wikipedia_images_anime.pickle', 'wb') as f:
    pickle.dump(merged_dict, f)

In [170]:
df = pd.DataFrame.from_dict({str(k): v for k, v in merged_dict.items()}, orient='index')
df = df.reset_index()
df['index'] = df['index'].astype(str)

In [172]:
df2['anime_id'] = df2['anime_id'].astype(str)

In [175]:
mal_df = df2.merge(df, left_on='anime_id', right_on = 'index', how = 'inner')
imdb_df = df2.merge(df, left_on='imdb_title_titleId', right_on = 'index', how = 'inner')

In [176]:
final_df = pd.concat([mal_df,imdb_df], ignore_index=True)

In [177]:
final_df

,imdb_title_title,imdb_title_titleId,anime_Rating,anime_English name,anime_Genres,anime_Other name,anime_Score,anime_Synopsis,anime_anime_id,imdb_name_basics_primaryName,...,Members,Duration,text,Name,anime_id,popular_recs,cf_recs,tokens,index,image_y
0,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,UNKNOWN,"Action, Drama",頭文字〈イニシャル〉D FOURTH STAGE,8.16,Takumi Fujiwara finally joins Ryousuke and Kei...,18,UNKNOWN,...,173710,27 min per ep,"Initial D (Japanese: 頭文字イニシャル D, Hepburn: Inis...",Initial D Fourth Stage,18,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[2608, 263, 245, 5258, 264, 265, 135, 71, 627,...","[initial, d, japanese, Tou, Wen, Zi, inishiyar...",18,[]
1,UNKNOWN,UNKNOWN,PG - Children,UNKNOWN,"Action, Sports",リングにかけろ１,6.39,"In order to fulfill their dead father's wish, ...",23,UNKNOWN,...,4581,25 min per ep,"Ring ni Kakero (Japanese: リングにかけろ, Hepburn: Ri...",Ring ni Kakero 1,23,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[48374, 51190, 9949, 9947, 20077, 16804, 30034...","[ring, ni, kakero, japanese, ringunikakero, he...",23,[https://upload.wikimedia.org/wikipedia/en/d/d...
2,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,D.C.~Da Capo~,"Drama, Romance",D.C.～ダ・カーポ～,6.72,Hatsunejima abounds in mysteries—one of which ...,62,UNKNOWN,...,74844,24 min per ep,"Da Capo (〜ダ・カーポ〜, Da Kāpo, commonly abbreviate...",D.C.: Da Capo,62,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[291, 2595, 3627, 5216, 4744, 2494, 3299, 119,...","[da, capo, ~, da*ka, po~, da, kapo, commonly, ...",62,[https://upload.wikimedia.org/wikipedia/en/5/5...
3,UNKNOWN,UNKNOWN,R+ - Mild Nudity,DearS,"Comedy, Romance, Sci-Fi, Ecchi",ディアーズ,6.56,"One year ago, a UFO containing 150 aliens cras...",63,UNKNOWN,...,145533,23 min per ep,Bears are carnivoran mammals of the family Urs...,DearS,63,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[241, 59, 487, 2494, 133, 3503, 53, 50, 4999, ...","[bear, carnivoran, mammal, family, ursidae, cl...",63,[https://upload.wikimedia.org/wikipedia/common...
4,UNKNOWN,UNKNOWN,PG-13 - Teens 13 or older,UNKNOWN,"Comedy, Drama, Romance",愛してるぜ ベイベ★★,7.44,Katakura Kippei is in every way a high school ...,102,UNKNOWN,...,101134,25 min per ep,"Aishiteruze Baby (愛してるぜ ベイベ★★, Aishiteruze Bei...",Aishiteruze Baby★★,102,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[1222, 3731, 53, 120, 2034, 394, 61, 101, 3470...","[aishiteruze, baby, Ai, shiteruze, beibe, aish...",102,[https://upload.wikimedia.org/wikipedia/en/2/2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11560,I've Always Liked You,tt5889338,UNKNOWN,I've Always Liked You,Romance,ずっと前から好きでした。～告白実行委員会～,7.22,Love is blooming at Sakuragaoka High School. N...,31245,"Kana Asumi, Gero, Yoshimi Narita, Momo Asakura...",...,220711,1 hr 3 min,Zutto Mae Kara Suki Deshita: Kokuhaku Jikkō Ii...,Zutto Mae kara Suki deshita. Kokuhaku Jikkou I...,31245,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[33036, 36220, 31414, 34822, 35434, 36470, 349...","[zutto, mae, kara, suki, deshita, kokuhaku, ji...",tt5889338,[https://upload.wikimedia.org/wikipedia/en/c/c...
11561,エクスドライバー Nina&Rei Danger Zone,tt1086281,UNKNOWN,UNKNOWN,"Action, Sci-Fi",エクスドライバー Nina&Rei Danger Zone,6.39,The story takes place on a world 100 years in ...,1733,"Wataru Tanaka, Michiko Neya, Mami Kingetsu, Kô...",...,4259,27 min,"éX-Driver (エクスドライバー, Ekusudoraibā) is an anime...",eX-Driver: Nina & Rei Danger Zone,1733,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[378, 377, 12351, 32385, 13177, 55186, 18047, ...","[ex, driver, ekusudoraiba-, ekusudoraiba, anim...",tt1086281,[]
11562,スクライド,tt0307427,UNKNOWN,s-CRY-ed,"Action, Adventure, Drama, Sci-Fi",スクライド,7.36,A strange environmental phenomenon 22 years ag...,168,"Lex Lang, Jamieson Price, Dave Wittenberg, Cri...",...,81088,24 min per ep,"s-CRY-ed (Japanese: スクライド, Hepburn: Sukuraido)...",s.CRY.ed,168,"[21, 1575, 1, 227, 431, 121, 226, 263, 2025, 47]","[400, 6, 567, 45, 205, 237, 90, 27, 392, 1]","[s, cry, ed, 